In [1]:
import glob
from collections import Counter, OrderedDict, defaultdict
import pysam
from pyBioInfo.Range import GRange
from pyBioInfo.IO.File import BedFile
from pyBioInfo.Utils import ShiftLoader

# Generated benchmark structural variant calls for HG001

In [2]:
infile = "GRCh38_HG001_PacBio_CCS/GRCh38_HG001_PacBio_CCS.sniffles2.filtered.vcf.gz"
outfile = "GRCh38_HG001_SV_benchmark_callsets.vcf"

records_lt50bp = []
records_gt10000bp = []
records_failed = []
records_imprecise = []
records_cluster = []

svs = [] # SVLEN >= 50 bp
min_cov = 15
min_hom_af = 0.9 

with pysam.VariantFile(infile) as f, open(outfile, "w+") as fw:
    for line in str(f.header).strip("\n").split("\n"):
        if line.startswith("##"):
            fw.write(line + "\n")
        else:
            fw.write('##FORMAT=<ID=IGT,Number=1,Type=String,Description="Original input genotype">\n')
            fw.write('##FORMAT=<ID=IPS,Number=1,Type=String,Description="Phase set for IGT">\n')
            fw.write('##FORMAT=<ID=PS,Number=1,Type=String,Description="Phase set for GT">\n')
            fw.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tHG001\n")
    for i, x in enumerate(f):
        if i % 1000 == 0:
            pass
            # print("Processed %d" % i)
        precise = x.info["PRECISE"]
        svtype = x.info["SVTYPE"]
        svlen = abs(x.info["SVLEN"])
        af = x.info["AF"]
        row = str(x).strip("\n").split("\t")
        if row[0] == "chrY":
            continue
        d = OrderedDict()
        for k, v in zip(row[8].split(":"), row[9].split(":")):
            d[k] = v
        d["IGT"] = d["GT"]
        dr = int(d["DR"])
        dv = int(d["DV"])
        cov = dr + dv
        FS = row[6].split(";")
        if FS[0] == "PASS":
            FS = []
        hp, ps, hp_support, ps_support, hp_filter, ps_filter = x.info["PHASE"]
        if len(FS) == 0:
            if svlen >= 50:
                if svlen > 10000:
                    records_gt10000bp.append(x)
                if cov < min_cov:
                    records_failed.append(x)
                    FS.append("COV_MIN")
                if not precise:
                    records_imprecise.append(x)
                    
                if d["GT"] == "1/1":
                    d["GT"] = "1|1"
                    d["PS"] = "0"
                    if af < min_hom_af:
                        records_failed.append(x)
                        FS.append("GT")
                elif d["GT"] == "0|1":
                    if hp_filter == "PASS" and ps_filter == "PASS":
                        assert hp == "1"
                        d["GT"] = "1|0"
                        d["PS"] = ps
                    else:
                        d["GT"] = "0/1"
                        d["PS"] = "."
                elif d["GT"] == "1|0":
                    if hp_filter == "PASS" and ps_filter == "PASS":
                        assert hp == "2"
                        d["GT"] = "0|1"
                        d["PS"] = ps
                    else:
                        d["GT"] = "0/1"
                        d["PS"] = "."
                elif d["GT"] == "0/1": 
                    d["PS"] = "."
                    if hp_filter == "PASS" and ps_filter == "PASS":
                        assert Fase
                elif d["GT"] == "1/0":
                    assert False
                else:
                    records_failed.append(x)
                    FS.append("GT")
                    d["PS"] = "."
            else:
                records_lt50bp.append(x)
                FS.append("SVLEN_MIN")
                if cov < min_cov:
                    FS.append("COV_MIN")
        else:
            if svlen >= 50:
                records_failed.append(x)
            else:
                records_lt50bp.append(x)
                FS.append("SVLEN_MIN")
        
        if svlen >= 50:
            sv = GRange(chrom=x.chrom, start=x.start, end=x.stop)
            sv.record = x
            svs.append(sv)
        
        if len(FS) == 0:
            FS = ["PASS"]
        row[6] = ";".join(FS)
        row[8] = ":".join(d.keys())
        row[9] = ":".join(d.values())
        
        fw.write("\t".join(row) + "\n")

svs.sort()
loader = ShiftLoader(svs)
for sv in svs:
    for sv2 in loader.fetch(chrom=sv.chrom, start=sv.start - 1000, end=sv.end + 1000):
        if sv2 is sv:
            continue
        records_cluster.append(sv.record)
        break
        
! bgzip -f {outfile}
! tabix -f -p vcf {outfile}.gz

In [3]:
print("SVs shorter than 50bp:", len(records_lt50bp))
print("SVs longer than 10kb:", len(records_gt10000bp))
print("SVs filter:", len(records_failed))
print("SVs imprecise:", len(records_imprecise))
print("SVs in cluster:", len(records_cluster))

SVs shorter than 50bp: 31655
SVs longer than 10kb: 48
SVs filter: 3413
SVs imprecise: 2558
SVs in cluster: 8150


In [4]:
sizes = "/home/chenzonggui/species/homo_sapiens/GRCh38.p13/GRCh38.canonical.genome.sizes"

outfile1 = "stratifications/GRC38_HG001_SV_lt50bp.bed"
outfile2 = "stratifications/GRC38_HG001_SV_lt50bp_slop100bp.bed"
with open(outfile1, "w+") as fw:
    for x in records_lt50bp:
        fw.write("\t".join(map(str, [x.chrom, x.start, x.stop])) + "\n")
! bedtools slop -i {outfile1} -g {sizes} -b 100 > {outfile2}

outfile1 = "stratifications/GRC38_HG001_SV_gt10000bp.bed"
outfile2 = "stratifications/GRC38_HG001_SV_gt10000bp_slop1000bp.bed"
with open(outfile1, "w+") as fw:
    for x in records_gt10000bp:
        fw.write("\t".join(map(str, [x.chrom, x.start, x.stop])) + "\n")
! bedtools slop -i {outfile1} -g {sizes} -b 1000 > {outfile2}

outfile1 = "stratifications/GRC38_HG001_SV_failed.bed"
outfile2 = "stratifications/GRC38_HG001_SV_failed_slop1000bp.bed"
with open(outfile1, "w+") as fw:
    for x in records_failed:
        fw.write("\t".join(map(str, [x.chrom, x.start, x.stop])) + "\n")
! bedtools slop -i {outfile1} -g {sizes} -b 1000 > {outfile2}

outfile1 = "stratifications/GRC38_HG001_SV_imprecise.bed"
outfile2 = "stratifications/GRC38_HG001_SV_imprecise_slop1000bp.bed"
with open(outfile1, "w+") as fw:
    for x in records_imprecise:
        fw.write("\t".join(map(str, [x.chrom, x.start, x.stop])) + "\n")
! bedtools slop -i {outfile1} -g {sizes} -b 1000 > {outfile2}

outfile1 = "stratifications/GRC38_HG001_SV_cluster.bed"
outfile2 = "stratifications/GRC38_HG001_SV_cluster_slop1000bp.bed"
with open(outfile1, "w+") as fw:
    for x in records_cluster:
        fw.write("\t".join(map(str, [x.chrom, x.start, x.stop])) + "\n")
! bedtools slop -i {outfile1} -g {sizes} -b 1000 > {outfile2}

In [5]:
# TandemRepeats, LowMappability and OtherDifficultRegions

prefix = "/home/chenzonggui/repositories/genome-stratifications/v3.1-genome-stratifications-GRCh38"

infile = prefix + "/LowComplexity/GRCh38_allTandemRepeats.bed.gz"
outfile = "stratifications/GRCh38_LowComplexity_gt500bp.bed"
! zcat {infile} | awk '$3-$2>500' > {outfile}

infile = prefix + "/mappability/GRCh38_lowmappabilityall.bed.gz"
outfile = "stratifications/GRCh38_LowMappability_gt500bp.bed"
! zcat {infile} | awk '$3-$2>500' > {outfile}

infile = prefix + "/OtherDifficult/GRCh38_allOtherDifficultregions.bed.gz"
outfile = "stratifications/GRCh38_OtherDifficultRegions_gt500bp.bed"
! zcat {infile} | awk '$3-$2>500' > {outfile}

In [6]:
for path in glob.glob("stratifications/*.bed"):
    ! sort -k1,1 -k2,2n {path} | bedtools merge -d 100 -i - | bgzip -c > {path}.gz
    ! tabix -f -p bed {path}.gz
    ! rm {path}

In [8]:
! zcat \
    stratifications/GRC38_HG001_SV_cluster_slop1000bp.bed.gz \
    stratifications/GRC38_HG001_SV_failed_slop1000bp.bed.gz \
    stratifications/GRC38_HG001_SV_lt50bp_slop100bp.bed.gz \
    stratifications/GRC38_HG001_SV_gt10000bp_slop1000bp.bed.gz \
    stratifications/GRC38_HG001_SV_imprecise_slop1000bp.bed.gz \
    stratifications/GRCh38_LowComplexity_gt500bp.bed.gz \
    stratifications/GRCh38_LowMappability_gt500bp.bed.gz \
    stratifications/GRCh38_OtherDifficultRegions_gt500bp.bed.gz \
    | grep -v chrY | sort -k1,1 -k2,2n | bedtools merge -d 100 -i - | bgzip -c > GRCh38_HG001_SV_benchmark_blacklist.bed.gz
! tabix -f -p bed GRCh38_HG001_SV_benchmark_blacklist.bed.gz

In [9]:
with BedFile("GRCh38_HG001_SV_benchmark_blacklist.bed.gz") as f:
    regions = [x for x in f]
    regions.sort()
print("Blacklist regions:", len(regions))

Blacklist regions: 89353


In [10]:
svs = []
with pysam.VariantFile("GRCh38_HG001_SV_benchmark_callsets.vcf.gz") as f:
    for x in f:
        svlen = abs(x.info["SVLEN"])
        if list(x.filter)[0] == "PASS" and svlen >= 50:
            sv = GRange(chrom=x.chrom, start=x.start, end=x.stop)
            sv.record = x
            svs.append(sv)
svs.sort()
print("Total SVs:", len(svs))

loader = ShiftLoader(regions)
tmp = []
for sv in svs:
    if len(list(loader.fetch(obj=sv))) == 0:
        tmp.append(sv)
svs1 = tmp
print("Filtered SVs:", len(svs1))

Total SVs: 21614
Filtered SVs: 10092


In [15]:
counter = defaultdict(int)
for sv in svs1:
    svtype = sv.record.info["SVTYPE"]
    gt = sv.record.samples["HG001"]["GT"]
    ps = sv.record.samples["HG001"]["PS"]
    counter[(svtype, gt, ps)] += 1
print("Number\tSVTYPE\tGT\tPS")
print("-" * 30)
for k, v in sorted(counter.items()):
    print(v, *k, sep="\t")

Number	SVTYPE	GT	PS
------------------------------
67	DEL	(0, 1)	.
1083	DEL	(0, 1)	0
1	DEL	(0, 1)	1160151
1	DEL	(0, 1)	32070669
1175	DEL	(1, 0)	0
1774	DEL	(1, 1)	0
127	INS	(0, 1)	.
1427	INS	(0, 1)	0
1	INS	(0, 1)	53795557
1404	INS	(1, 0)	0
3032	INS	(1, 1)	0
